In [ ]:
# Setup commands you'll need to configure the environment
#conda activate /Users/sergenes/opt/anaconda3
#/Users/sergenes/opt/anaconda3/bin/conda install -p /Users/sergenes/opt/anaconda3 anthropic -y
#pip install anthropic
#pip install ai21

#curl "https://awscli.amazonaws.com/AWSCLIV2.pkg" -o "AWSCLIV2.pkg"
#sudo installer -pkg AWSCLIV2.pkg -target /

#aws --version
# aws-cli/2.14.1 Python/3.11.6 Darwin/23.0.0 exe/x86_64 prompt/off

#aws configure
#aws sts get-caller-identity
# {
#     "UserId": "12345678901",
#     "Account": "12345678901",
#     "Arn": "arn:aws:iam::12345678901:root"
# }
#vi ~/.aws/credentials
#aws bedrock list-foundation-models --by-provider meta --query "modelSummaries[*].modelId"

In [97]:
import boto3
import json

bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name='us-east-1')
accept = 'application/json'
contentType = 'application/json'

In [5]:
# Check that the boto3 is configured, get_foundation_model should return info
bedrock = boto3.client(service_name='bedrock')

bedrock.get_foundation_model(modelIdentifier='meta.llama2-13b-chat-v1')

{'ResponseMetadata': {'RequestId': 'fee96247-1f30-449b-9c9e-9c5cf70ef96b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 30 Nov 2023 15:16:12 GMT',
   'content-type': 'application/json',
   'content-length': '383',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'fee96247-1f30-449b-9c9e-9c5cf70ef96b'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/meta.llama2-13b-chat-v1',
  'modelId': 'meta.llama2-13b-chat-v1',
  'modelName': 'Llama 2 Chat 13B',
  'providerName': 'Meta',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND']}}

In [98]:
def get_bedrock_meta_completion(bprompt, bmodel="meta.llama2-13b-chat-v1"):
    print(bmodel)
    body = json.dumps({
        "prompt": bprompt,
        "max_gen_len": 512,
        "temperature": 0.1,
        "top_p": 0.9,
    })

    response = bedrock_runtime.invoke_model(body=body, modelId=bmodel, accept="*/*", contentType=contentType)

    response_body = json.loads(response.get('body').read())
    return response_body

In [99]:
def get_bedrock_cohere_completion(bprompt, bmodel="cohere.command-text-v14"):
    body = json.dumps({
        "prompt": bprompt,
        "max_tokens": 2048,
        "temperature": 0.1
    })

    response = bedrock_runtime.invoke_model(body=body, modelId=bmodel, accept=accept, contentType=contentType)

    response_body = response.get('body').read()
    data = json.loads(response_body)
    text = data['generations'][0]['text']
    return text

In [100]:
import ai21

boto_session = boto3.Session(region_name="us-east-1")


def get_bedrock_ai21_completion(bprompt, bmodel="ai21.j2-ultra-v1"):
    response = ai21.Completion.execute(
        destination=ai21.BedrockDestination(
            model_id=bmodel,
            boto_session=boto_session,
        ),
        prompt=bprompt,
        numResults=1,
        maxTokens=2048,
        temperature=0.2
    )
    text = response.completions[0].data.text
    return text

In [101]:
def get_bedrock_titan_completion(bprompt, bmodel="amazon.titan-text-express-v1"):
    body = json.dumps({
        "inputText": bprompt,
        "textGenerationConfig": {
            "maxTokenCount": 4048,
            "stopSequences": [],
            "temperature": 0.2,
            "topP": 0.8
        }
    })

    response = bedrock_runtime.invoke_model(body=body, modelId=bmodel, accept=accept, contentType=contentType)
    response_body = response.get('body').read()
    data = json.loads(response_body)
    text = data['results'][0]['outputText']
    return text

In [102]:
prompt_json = """
You are an assistant that specializes in travel planning. You are communicating with the LLM agent so return the JSON array only! You task is to plan a walking tour in Boston where tourists would like to visit 5 landmark places on the Freedom Trail.

For each place, please provide the following details in a plain English:
- Name;
- A brief description of the place with a bit of its history (about five sentences);
- location (latitude and longitude).

The list should be optimized for a walking tour, meaning the places should be ordered in the way one would visit them walking along the Freedom Trail.

The information should be provided in the following valid, JSON format:

{
  "place_id": "<id>",
  "name": "<name>",
  "description": "<description>",
  "location": "<latitude,longitude>"
}

Make sure you return the only json as a result, no comments, no surrounding text, json array only!
"""

In [111]:
def proofreading_prompt(text):
    return f"""
    You are tasked with proofreading and refining the text enclosed within triple backticks. Please follow these instructions carefully:
    1. Language Consistency:
    Please, determine the original language and use it consistently throughout all tasks such as edits, feedback, or explanations must be in the same.
    Correct syntax, punctuation, and other errors.
    2. Feedback Format:
    After editing, provide a clear breakdown of the changes you've made and explain your reasons in original language.
    3. Output:
    Present the output in the following JSON format:
    {{
      "language": "<string, original language>",
      "result": "<plain string without backticks, edited text>",
      "changes": "<string, breakdown of your changes and the reasons in original language>"
    }}
    Text for editing:
    ```{text}```
    Make sure you return the only json as a result, no comments, no surrounding text, no input prompt, json object only!
    """

In [104]:
def print_values(name, model, running_time, response):
    print(f"======={name} API ({model}) Begins========")
    print(f"running_time=>{running_time:.2f}")
    print(response)
    print(f"======={name} API ({model}) Ends========")

In [105]:
# Excluding the embedding models.
# Also, the Anthropic and Stability AI models are excluded from the list
# as first is tested in another set and the second is Image generating model provider
providers = {
    "ai21": ["ai21.j2-mid-v1", "ai21.j2-ultra-v1"],
    "amazon": ["amazon.titan-tg1-large",
               # "amazon.titan-image-generator-v1",
               # "amazon.titan-embed-g1-text-02",
               "amazon.titan-text-lite-v1",
               "amazon.titan-text-express-v1",
               # "amazon.titan-embed-text-v1",
               # "amazon.titan-embed-image-v1"
               ],
    "meta": ["meta.llama2-13b-chat-v1",
             "meta.llama2-70b-chat-v1",
             "meta.llama2-13b-v1",
             # "meta.llama2-70b-v1" - I do not have access
             ],
    "cohere": ["cohere.command-text-v14",
               "cohere.command-light-text-v14",
               # "cohere.embed-english-v3",
               # "cohere.embed-multilingual-v3"
               ]
}

In [106]:
import time

for model in providers["ai21"]:
# model = providers["ai21"][0]
    start = time.time()
    response = get_bedrock_ai21_completion(prompt_json, model)
    running_time = time.time() - start
    print_values("AI21 Labs Bedrock", model, running_time, response)

# I wasn't able to make Meta models work in the Bedrock, tho through Perplexity API it works good!
# for model in providers["meta"]:
#     # model = providers["meta"][2]
#     start = time.time()
#     response = get_bedrock_meta_completion(prompt_json, model)
#     running_time = time.time() - start
#     print_values("Meta Bedrock", model, running_time, response)
#
for model in providers["amazon"]:
# model = providers["amazon"][2]
    start = time.time()
    response = get_bedrock_titan_completion(prompt_json, model)
    running_time = time.time() - start
    print_values("Amazon Bedrock", model, running_time, response)

for model in providers["cohere"]:
# model = providers["cohere"][1]
    start = time.time()
    response = get_bedrock_cohere_completion(prompt_json, model)
    running_time = time.time() - start
    print_values("Cohere Bedrock", model, running_time, response)

=======AI21 Labs Bedrock API (ai21.j2-mid-v1) Begins========
running_time=>5.99
[
  {
    "place_id": "1",
    "name": "Old State House",
    "description": "The Old State House is historic building in Boston, Massachusetts. It was constructed in 1713 and is the oldest surviving public building in Boston. The Old State House is listed on the National Register of Historic Places and is a National Historic Landmark. It is currently a museum operated by the Bostonian Society.",
    "location": "42.350255,-71.070256"
  },
  {
    "place_id": "2",
    "name": "Faneuil Hall",
    "description": "Faneuil Hall is a historic building in Boston, Massachusetts. It was constructed in 1740 and is a National Historic Landmark. Faneuil Hall is a popular destination for tourists and locals alike, and is often used for events and concerts.",
    "location": "42.360255,-71.070256"
  },
  {
    "place_id": "3",
    "name": "Quincy Market",
    "description": "Quincy Market is a historic building in Bosto

In [113]:
# More simple task to proofread text
import time

text = "Their is a few incorrect use's of grammar and misspelled words' in this sentence for you too proofread."

for model in providers["ai21"]:
    start = time.time()
    response = get_bedrock_ai21_completion(proofreading_prompt(text), model)
    running_time = time.time() - start
    print_values("AI21 Labs Bedrock", model, running_time, response)

for model in providers["amazon"]:
    start = time.time()
    response = get_bedrock_titan_completion(proofreading_prompt(text), model)
    running_time = time.time() - start
    print_values("Amazon Bedrock", model, running_time, response)

for model in providers["cohere"]:
    start = time.time()
    response = get_bedrock_cohere_completion(proofreading_prompt(text), model)
    running_time = time.time() - start
    print_values("Cohere Bedrock", model, running_time, response)

=======AI21 Labs Bedrock API (ai21.j2-mid-v1) Begins========
running_time=>1.33

    {
      "language": "English",
      "result": "Their are a few incorrect use's of grammar and misspelled words' in this sentence for you too proofread.",
      "changes": "Changed 'use's' to 'are' and corrected the spelling of 'too'."
    }
=======AI21 Labs Bedrock API (ai21.j2-mid-v1) Ends========
=======AI21 Labs Bedrock API (ai21.j2-ultra-v1) Begins========
running_time=>2.94

{
  "language": "English",
  "result": "Their is a few incorrect uses of grammar and misspelled words in this sentence for you too proofread.",
  "changes": "Replaced 'is' with 'are' to match the plural form of 'few'. Replaced 'too' with 'to' because it is the correct form of the word in this context. Added apostrophe in the word 'misspelled' because it is the possessive form of the word (misspelled words). Added an apostrophe in the word 'sentence' because it is the possessive form of the word (sentence's)."
}
=======AI21 La